# From OMOP to IDEA4RC Cancer Episode

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

curOMOP = conn.cursor()

In [ ]:
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

# Doubts:

How is the hash supposed to be created so it hashes to the same cancer episodes, with wrong diagnosis? how do we know if they are the same with OMOP?

Version?

## Overarching Episode Creation

We will take an overarching episode and all the other cancer episodes that we have

In [ ]:
query= """
    SELECT episode_id, person_id, episode_start_date, episode_concept_id, episode_object_concept_id,
    episode_type_concept_id, parent_episode_id FROM episode
    WHERE episode_concept_id=32533 AND episode_type_concept_id=32879;
    """
curOMOP.execute(query)
episodes = curIDEA.fetchall()

insert= """
    INSERT INTO cancer_episode (id,patient, start_date) VALUES (%s,%s, %s);
"""
dictKeys={}

for episode in episodes:
    episode_id = episode[0]
    patient = episode[1]
    start_date = episode[2]
    ep_occurrence_id=datetime.now().strftime("%Y%m%d%H%M%S")+ str(uuid4())
    curIDEA.execute(insert,(ep_occurrence_id,patient,start_date))
    dictKeys[episode_id] = ep_occurrence_id
    curIDEA.commit()

### From Procedure Occurrence to Cancer Episode

In [ ]:
query= """
    SELECT id
    FROM patient;
"""

query2= """
    SELECT procedure_date, procedure_concept_id, provider_id, episode_id 
    FROM procedure_occurrence INNER JOIN episode_event ON procedure_occurrence.procedure_occurrence_id = episode_event.event_id
    WHERE patient_id = %s
    AND 1147082=episode_event_field_concept_id
    AND procedure_concept_id IN (4171863,4321878,4321986,4228202,4279903);
"""
curOMOP.execute(query)

results = curIDEA.fetchall()

for row in results:
    curOMOP.execute(query2, (row))
    results2 = curOMOP.fetchall()
    for row2 in results2:
        update= """
            UPDATE cancer_episode SET date_of_diagnosis = %s, type_of_biopsy = %s, biopsy_done_by = %s
            WHERE patient = %s AND id= %s; 
        """
        curIDEA.execute(update, (row[0], row2[0], row2[1], row2[2], dictKeys.get(row2[3])))

conn2.commit()
conn.commit()
curIDEA.close()
curOMOP.close()

### From Condition Occurrence to Cancer Episode

for histology_adenocarcinoma, what should the code be for 'LOBULAR_CARCINOMA','SALIVARY_DUCT_CARCINOMA','MAMMARY_SECRETORY_CARCINOMA','MYOEPITHELIAL_CARCINOMA','EX_PLEOMORPHIC_ADENOCARCINOMA','SEBACEOUS_LYMPHADENOCARCINOMA', 'SALIVARY_GLAND','NASOPHARYNX_PAPILLARY_ADENOCARCINOMA'???. Also we have 10 codes which we don't find in the initdb.sql

histology_neuroendocrine we have 2 codes we dont find in the initdb.sql

histology_odontogenic_carcinoma 1 code we dont find in the db, and 'ODONTOGENIC_CARCINOMA','AMELOBLASTIC_CARCINOMA','DEDIFFERENTIATED_INTRAOSSEOUS_AMELOBLASTIC_CARCINOMA','DEDIFFERENTIATED_PERIPHERAL_AMELOBLASTIC_CARCINOMA','PRIMARY_INTRAOSSEOUS_CARCINOMA','INTRAOSSEOUS_CARCINOMA_DEVELOPED_ON_ODONTOGENIC_CYST'

I believe these columns are missing: Histology subgroup Adenosquamous carcinoma, Histology subgroup Teratocarcinosarcoma, Histology subgroup NUT carcinoma, Histology subgroup HPV-related Multiphenotypic Sinonasal Carcinoma, Histology subgroup Olfactory neuroblastoma (esthesioneuroblastoma; esthesioneurocytoma; esthesioneuroepithelioma; olfactory placode tumor), Endometrial stromal and related tumours, Parotid gland, Submandibular gland, Sublingual gland, Middle ear, Carcinoma/carcinoma undifferentiated

Other subsite??? We do not have a vocabulary.

histology_subgroup_adipocytic_tumours: what to do with 4021372, and what the custom code for MYXOID_PLEOMORPHIC_LIPOSARCOMA is

histology_subgroup_fmt_tumours: 'SOLITARY_FMT_MALIGNANT','INFANTILE_FIBROSARCOMA', and code 37116997

vascular_tumours: what to do with code 4030139

oropharynx_subsite: what to do with code 4327174

oral_cavity_subsite: what to do with codes 44498031, 4284551, 44498052 and VENTRAL_SURFACE_OF_TONGUE

lip_subsite: MUCOSA_LOWER_LIP?





In [ ]:
histology_squamous = {
    4078953: 'KERATINIZING_SQUAMOUS_CELL_CARCINOMA',
    4166826: 'NON_KERATINIZING_SQUAMOUS_CELL_CARCINOMA',
    4206785: 'SCHNEIDERIAN_CARCINOMA',
    4009590: 'SPINDLE_CELLSQUAMOUS_CELL_CARCINOMA',
    4247661: 'LYMPHOEPITHELIAL_CARCINOMA',
    4029973: 'BASALOID_SQUAMOUS_CELL_CARCINOMA',
    4233949: 'VERRUCOUS_SQUAMOUS_CELL_CARCINOMA',
    4191609: 'PAPILLARY_SQUAMOUS_CELL_CARCINOMA',
    37156145: 'SQUAMOUS_CELL_CARCINOMA',
    37204531: 'SQUAMOUS_CELL_CARCINOMA_HPV_POSITIVE',
    37204532: 'SQUAMOUS_CELL_CARCINOMA_HPV_NEGATIVE'
}

histology_adenocarcinoma = {
    4097305: '???',
    37152526: '???',
    4050978: 'CYSTOADENOCARCINOMA',
    4247921: 'MUCINOUS',
    4238334: 'CERUMINOUS',
    4172953: '???',
    4022895: 'ADENOID_CYSTIC_CARCINOMA',
    4253608: 'MUCOEPIDERMOID_CARCINOMA',
    4030121: 'POLYMORPHOUS',
    4164740: 'ACINAR_CELL_CARCINOMA',
    4148292: 'CLEAR_CELL_CARCINOMA',
    4146684: 'BASAL_CELL_ADENOCARCINOMA',
    4221403: '???',
    4224593: '???',
    4323699: '???',
    4029680: '???',
    4321002: '???',
    4066512: '???',
    4182993: 'SEBACEOUS_ADENOCARCINOMA',
    4271564: 'CARCINOSARCOMA',
    4212379: 'OXYPHILIC_CARCINOMA',
    4050978: '???'
}

histology_neuroendocrine= {
    36714029:'SMCC',
    37018672:'POORLY_DIFFERENTIATED_NEUROENDOCRINE_CARCINOMA',
    4029971:'LARGE_CELL_NEUROENDOCRINE_CARCINOMA',
    4133828:'???',
    4133297:'???'
}
histology_odontogenic_carcinoma= {
    4098585:'???',
    37116966:'CLEAR_CELL_ODONTOGENIC_CARCINOMA',
    4005818: 'GHOST_CELL_ODONTOGENIC_CARCINOMA'
}

histology_snuc= {
    4243327:'SNUC'
}

histology_subgroup_adipocytic_tumours={
    4021372:???,
    4102790:'LIPOSARCOMA_WELL_DIFF',
    4284833:'DEDIFFERENTIATED_LIPOSARCOMA',
    4101123:'MYXOID_LIPOSARCOMA',
    4010104:'PLEOMORPHIC_LIPOSARCOMA',
}

histology_subgroup_fmt_tumours={
    45766452:'PALMAR_PLANTAR_FMT',
    4264204:'AGGRESSIVE_FIBROMATOSIS',
    45766468:'LIPOFIBROMATOSIS',
    4030132:'GIANT_CELL_FIBROBLASTOMA',
    4295283:'DERMATOFIBROSARCOMA_PROTUBERANS',
    4030130:'SOLITARY_FIBROUS_TUMOR',
    37116997:???,
    4021373:'MYOFIBROBLASTIC_TUMOUR',
    45766454:'LOW_GRADE_MYOFIBROBLASTIC_SARCOMA',
    4029679:'CELLULAR_FIBROMA',
    45766450:'MYXOINFLAMMATORY_FIBROBLASTIC_SARCOMA',
    4202374:'FIBROSARCOMA',
    45771359:'MYXOFIBROSARCOMA',
    45771361:'LOW_GRADE_FIBROMYXOID_SARCOMA',
    45766456:'SCLEROSING_EPITHELIOID_FIBROSARCOMA'
}

histology_subgroupsocalled_fibrohistiocytic_tumours={
    4029525:'PLEXIFORM_FIBROHISTIOCYTIC_TUMOR',
    4218898:'GIANT_CELL_TUMOR_SOFT_PARTS',
    4030140:'TENOSYNOVIAL_GIANT_CELL_TUMOR',
    45766525:'TENOSYNOVIAL_GIANT_CELL_TUMOR_DIFFUSE',
    4029655:'MALIGNANT_TENOSYNOVIAL_GIANT_CELL_TUMOR'
}

vascular_tumours={
    4207381:'EPITHELIOID_HAEMANGIOENDOTHELIOMA',
    3661612:'ANGIOSARCOMA',
    4281384:'HEMANGIOENDOTHELIOMA',
    45766493:'RETIFORM_HAEMANGIOENDOTHELIOMA',
    4030139:???,
    4030138:'PAPILLARY_INTRALYMPH_ANGIOENDOTHELIOMA',
    45766494:'COMPOSITE_HAEMANGIOENDOTHELIOMA',
    45766492:'PSEUDOMYOGENIC_HEMANGIOENDOTHELIOMA'
}

pericytic_perivascular_tumours={
    40571984:'GLOMUS_TUMOUR'
}

smooth_muscle_tumours={
    4327493:'SMOOTH_MUSCLE_TUMOUR',
    607803:'EPITHELIOID_STUMP',
    600675:'MYXOID_STUMP',
    4029023:'METASTASIZING_LEIOMYOMA',
    4201415:'LEIOMYOSARCOMA',
    4173148:'EPITHELIOID_LEIOMYOSARCOMA',
    4059632:'MYXOID_LEIOMYOSARCOMA',
    4029023:'METASTASIZING_LEIOMYOMA'
}

skeletal_muscle_tumours={
    4031038:'EMBRYONAL_RHABDOMYOSARCOMA',
    4272931:'ALVEOLAR_RHABDOMYOSARCOMA',
    4298312:'PLEOMORPHIC_RHABDOMYOSARCOMA',
    4029528:'SPLINDLE_CELL_RHABDOMYOSARCOMA',
    4029024:'RHABDOMYOSARCOMA_GANGLIONIC_DIFF'
}

chondro_osseous_tumours={
    4298240:'EXTRASKELETAL_OSTEOSARCOMA'
}

peripheral_nerve_sheath_tumours={
    4062426:'MALIGNANT_PERI_NERVE_SHEATH_TUMOR',
    4102399:'MELANOTIC_MALIGNANT_PERI_NERVE_SHEATH_TUMOR',
    4005360:'GRANULAR_CELL_TUMOUR',
    4027842:'GRANULAR_CELL_TUMOUR_MALIGN'
}

tumours_of_uncertain_differentiation={
    4096931:'ATYPICAL_FIBROUS_HISTIOCYTOMA',
    4029526:'ANGIOMATOID_FIBROUS_HISTIOCYTOMA',
    45766448:'PLEOMORPHIC_HYALINIZING_ANGIECTATIC_TUMOUR',
    4268491:'SYNOVIAL_SARCOMA',
    4244886:'EPITHELIOID_SARCOMA',
    4339208:'ALVEOLAR_SOFT_PART_SARCOMA',
    4155072:'CLEAR_CELL_SARCOMA',
    4328092:'MYXOID_CHONDROSARCOMA',
    4029522:'DESMOPLASTIC_SMALL_ROUND_CELL_TUMOR',
    4307938:'MALIGNANT_RHABDOID_TUMOR',
    45771358:'PERIVASCULAR_EPITHELIOID_TUMOR_BENIGN',
    45766447:'PERIVASCULAR_EPITHELIOID_TUMOR_MALIGNANT',
    4304780:'PERIVASCULAR_EPITHELIOID_CELL_TUMOR',
    45766495:'INTIMAL_SARCOMA',
    4029021:'UNDIFFERENTIATED_SARCOMA',
    4239519:'MYOEPITHELIOMA',
    4028565:'OSSIFYING_FIBROMYXOID_TUMOUR',
    45766485:'PHOSPHATURIC_MESENCHYMAL_TUMOUR_MALIGNANT',
    4239956:'ANGIOMYXOMA',                                            #Is this the aggressive form?
    4029526:'ANGIO_FIBR_HISTIOCYT'
}

undif_smallrcel_sarc_bon_and_sof_tis={
    45766528:'EWING_SARCOMA'
}

miscellanious_mesenchimal_tumors={
    45771357:'UTERINE_TUMOR',
    4040991:'BENIGN_PHYLLODES_TUMOR',
    4337106:'BORDERLINE_PHYLLODES_TUMOR',
    4337106:'PHYLLODES_TUMOUR_MALIGNANT',
    4028710:'FOLLICULAR_DENDRITIC_CELL_SARCOMA',
    4029172:'HISTIOCYTIC_SARCOMA',
    4028709:'INTERDIGITATING_DENDRITIC_CELL_SARCOMA',
    4029173:'LANGHERANS_CELL_SARCOMA',
    42872917:'FIBROBLASTIC_RETICULAR_CELL_TUMOUR',
    37311513:'BIPHENOTYPIC_SINONASAL_SARCOMA'
}

mixed_epithelial_and_mesenchymal_tumours={
    4135215:'ADENOSARCOMA'
}
nasal_cavity_and_paranasal_sinuses_subsite={
    4088327:'NASAL_CAVITY',
    4050631:'MAXILLARY_SINUS',
    4182234:'ETHMOID_SINUS',
    4204534:'FRONTAL_SINUS',
    4097172:'SPHENOID_SINUS'
}

nasopharynx_subsite={
    4068978:'SUP_WALL_NASOPHARYNX',
    4146741:'POST_WALL_NASOPHARYNX',
    4323979:'LAT_WALL_NASOPHARYNX',
    4220236:'ANT_WALL_NASOPHARYNX'
}

hypopharynx_subsite={
    4147741:'POSTCRICOID_REGION',
    4262980:'HYPOPHARYNGEAL_ASPECT_ARYE_FOLD',
    4102600:'POST_WALL_HYPOPHARYNX',
    4268741:'PIRIFORM_SINUS'
}

oropharynx_subsite={
    4169910:'BASE_TONGUE',
    4177005:'SOFT_PALATE_NOS',
    4095284:'UVULA',
    4138234:'TONSILLAR_FOSSA',
    4068841:'LINGUAL_TONSIL',
    4065073:'TONSILLAR_PILLAR',
    4173123:'VALLECULA',
    4283238:'ANTERIOR_SURFACE_OF_EPIGLOTTIS',
    4078066:'LATERAL_WALL_OROPHARYNX',
    4045435:'POSTERIOR_WALL_OROPHARYNX',
    4235649:'BRANCHIAL_CLEFT',
    4327174:???
}

larynx_subsite={
    4047227:'GLOTTIS',
    4001049:'SUPRAGLOTTIS',
    4002724:'SUBGLOTTIS',
    4279711:'LARYNGEAL_CARTILAGE'
}

oral_cavity_subsite={
    4284551:???,obs_event_field_concept_id
    4077986:'BORDER_TONGUE',
    4172794:'DORSAL_SURFACE_TONGUE',                           #Is this correct?
    44498031:???,
    4047029:'UPPER_GUM',
    4210919:'LOWER_GUM',
    4181419:'ANTERIOR_FLOOR_OF_MOUTH',
    4103066:'LATERAL_FLOOR_OF_MOUTH',
    44498040:'OVERLAPPING_LESION_OF_FLOOR_OF_MOUTH',
    4234542:'HARD_PALATE',
    4057160:'CHEEK_MUCOSA',
    4144069:'VESTIBULE_OF_MOUTH',
    4312102:'RETROMOLAR_AREA',
    44498052:???
}

lip_subsite={
    4134589:'EXTERNAL_UPPER_LIP',
    4027546:'EXTERNAL_LOWER_LIP',
    4027545:'EXTERNAL_LIP',
    4086480:'MUCOSA_UPPER_LIP',
    4245168:'MUCOSA_LIP',
    4307358:'COMMISURE_LIP',
}

upper_and_lower_limbs_subsite={
    4310976:'HAND_STRUCTURE',
    4220073:'WRIST_REGION_STRUCTURE',
    4032953:'FOREARM_STRUCTURE',
    4027910:'ELBOW_REGION_STRUCTURE',
    4348994:'ANTECUBITAL_FOSSA_SURFACE_REGION',
    4165266:'UPPER_ARM_STRUCTURE',
    4211098:'FOOT_STRUCTURE',
    4142359:'ANKLE_REGION_STRUCTURE',
    4107527:'LOWER_LEG_STRUCTURE',
    4217806:'KNEE_REGION_STRUCTURE',
    4138531:'POPLITEAL_FOSSA_STRUCTURE',
    4090445:'UPPER_LEG',
    4291935:'DELTOID',
    4088464:'SUPRACLAVICULAR',
    42605854:'PERISCAPULAR_REGION',
    4136969:'TRAPEZIUS',
    4109459:'GROIN',
    4263335:'BUTTOCK',
    4096078:'PECTORAL',
    4157836:'AXILLA'
}

trunk_wall_subsite={
    4088614:'JOINT_STRUCT_THORACIC_WALL',
    4244412:'ANTERIOR_ABDOMINAL_WALL_STRUCTURE',
    4330865:'POSTERIOR_ABDOMINAL_WALL_STRUCTURE'
}

intra_abdominal_subsite= {
    4220536:'RETROPERITONEAL_COMP_STRUCT',
    4271699:'INFERIOR_VENA_CAVA_STRUCTURE',
    4193451:'ILIAC_VESSEL',
    4154880:'STRUCTURE_OF_RENAL_ARTERY',
    4157844:'GONADAL_ARTERY',
    4314264:'STRUCTURE_OF_MESENTERIC_ARTERY',
    4094242:'RENAL_PELVIS_STRUCTURE',
    4229753:'MAJOR_PELVIS_STRUCTURE',
    4075991:'REGION_OF_ESOPHAGUS',
    4288283:'STOMACH',
    4302112:'DUODENAL_STRUCTURE',
    4065077:'JEJUNAL_STRUCTURE',
    4316356:'STRICTURE_OF_ILEUM',
    4215634:'COLON_STRUCTURE',
    4144541:'RECTUM_STRUCTURE',
    4233095:'MESENTERY'
}

intra_thoracic_subsite={
    4213162:'LUNG_STRUCTURE',
    4217142:'HEART_STRUCTURE',
    4150673:'PLEURA_STRUCTURE',
    4088610:'REGION_OF_MEDIASTINUM',
    4167648:'SUPERIOR_VENA_CAVA_STRUCTURE',
    4001033:'AORTA_PART',
    4035327:'STRICTURE_OF_PULMONARY_VESSEL',
    4272388:'STRUCTURE_OF_SUBCLAVIAN_ARTERY'
}

genito_urinary_subsite={
    4047078:'BILATERAL_ADRENAL_GLANDS',
    4271678:'KIDNEY_STRUCTURE',
    4078617:'REGION_OF_URETER',
    4232481:'URINARY_BLADDER_STRUCTURE',
    4034365:'STRUCTURE_OF_LOBE_OF_PROSTATE',
    4277442:'SEMINAL_VESCICLE_STRUCTURE',
    4172591:'SPERMATIC_CORD_STRUCTURE',
    4238088:'TESTIS_STRUCTURE',
    4077311:'PENILE_STRUCTURE',
    4149192:'UTERINE_STRUCTURE',
    4049448:'OVARIAN_STRUCTURE',
    4151444:'FALLOPIAN_TUBE_STRUCTURE',
    4296022:'VAGINAL_STRUCTURE',
    4166066:'VULVAL_STRUCTURE'
}

head_and_neck_subsite={
    4122633:'FACE_NECK_STRUCTURE'
}

breast_subsite={4298444:'BREAST_STRUCTURE'}

est_and_related_tumours={
    4290926:'EST_SARCOMA_HIGH_GRADE',
    4028557:'EST_SARCOMA_LOW_GRADE'
}

query= """
    SELECT id
    FROM patient;
"""

query2= """
    SELECT condition_concept_id, condition_start_date, condition_type_concept_id
    FROM omopcdm.condition_occurrence WHERE patient_id = %s;
"""
curOMOP.execute(query)

results = curIDEA.fetchall()

def update_cancer_episode(row, histology_dict, histology_column):
    #Updates the cancer_episode table with the corresponding histology type.
    insert_query = f"""
        UPDATE cancer_episode
        SET {histology_column} = %s
        WHERE patient_id = %s AND date_of_diagnosis = %s;
    """
    value = histology_dict.get(row2[0])
    if value:
        curIDEA.execute(insert_query, (value, row[0], row2[1]))

for row in results:
    curIDEA.execute(query2, (row))
    results2 = curIDEA.fetchall()
    for row2 in results2:
        if row2[0] in histology_squamous.keys():
            update_cancer_episode(row, histology_squamous, 'histology_squamous')
        elif row2[0] in histology_adenocarcinoma.keys():
            update_cancer_episode(row, histology_adenocarcinoma, 'histology_adenocarcinoma')
        elif row2[0] in histology_neuroendocrine.keys():
            update_cancer_episode(row, histology_neuroendocrine, 'histology_neuroendocrine')
        elif row2[0] in histology_odontogenic_carcinoma.keys():
            update_cancer_episode(row, histology_odontogenic_carcinoma, 'histology_odontogenic_carcinoma')
        elif row2[0] in histology_snuc.keys():
            update_cancer_episode(row, histology_snuc, 'histology_snuc')
        elif row2[0] in histology_subgroup_adipocytic_tumours.keys():
            update_cancer_episode(row, histology_subgroup_adipocytic_tumours, 'histology_subgroup_adipocytic_tumours')
        elif row2[0] in histology_subgroup_fmt_tumours.keys():
            update_cancer_episode(row, histology_subgroup_fmt_tumours, 'histology_subgroup_fmt_tumours')
        elif row2[0] in histology_subgroupsocalled_fibrohistiocytic_tumours.keys():
            update_cancer_episode(row, histology_subgroupsocalled_fibrohistiocytic_tumours, 'histology_subgroupsocalled_fibrohistiocytic_tumours')
        elif row2[0] in vascular_tumours.keys():
            update_cancer_episode(row, vascular_tumours, 'vascular_tumours')
        elif row2[0] in pericytic_perivascular_tumours.keys():
            update_cancer_episode(row, pericytic_perivascular_tumours, 'pericytic_perivascular_tumours')
        elif row2[0] in smooth_muscle_tumours.keys():
            update_cancer_episode(row, smooth_muscle_tumours, 'smooth_muscle_tumours')
        elif row2[0] in skeletal_muscle_tumours.keys():
            update_cancer_episode(row, skeletal_muscle_tumours, 'skeletal_muscle_tumours')
        elif row2[0] in chondro_osseous_tumours.keys():
            update_cancer_episode(row, chondro_osseous_tumours, 'chondro_osseous_tumours')
        elif row2[0] in peripheral_nerve_sheath_tumours.keys():
            update_cancer_episode(row, peripheral_nerve_sheath_tumours, 'peripheral_nerve_sheath_tumours')
        elif row2[0] in tumours_of_uncertain_differentiation.keys():
            update_cancer_episode(row, tumours_of_uncertain_differentiation, 'tumours_of_uncertain_differentiation')
        elif row2[0] in undif_smallrcel_sarc_bon_and_sof_tis.keys():
            update_cancer_episode(row, undif_smallrcel_sarc_bon_and_sof_tis, 'undif_smallrcel_sarc_bon_and_sof_tis')
        elif row2[0] in miscellanious_mesenchimal_tumors.keys():
            update_cancer_episode(row, miscellanious_mesenchimal_tumors, 'miscellanious_mesenchimal_tumors')
        elif row2[0] in mixed_epithelial_and_mesenchymal_tumours.keys():
            update_cancer_episode(row, mixed_epithelial_and_mesenchymal_tumours, 'mixed_epithelial_and_mesenchymal_tumours')
        elif row2[0] in nasal_cavity_and_paranasal_sinuses_subsite.keys():
            update_cancer_episode(row, nasal_cavity_and_paranasal_sinuses_subsite, 'nasal_cavity_and_paranasal_sinuses_subsite')
        elif row2[0] in nasopharynx_subsite.keys():
            update_cancer_episode(row, nasopharynx_subsite, 'nasopharynx_subsite')
        elif row2[0] in hypopharynx_subsite.keys():
            update_cancer_episode(row, hypopharynx_subsite, 'hypopharynx_subsite')
        elif row2[0] in oropharynx_subsite.keys():
            update_cancer_episode(row, oropharynx_subsite, 'oropharynx_subsite')
        elif row2[0] in larynx_subsite.keys():
            update_cancer_episode(row, larynx_subsite, 'larynx_subsite')
        elif row2[0] in oral_cavity_subsite.keys():
            update_cancer_episode(row, oral_cavity_subsite, 'oral_cavity_subsite')
        elif row2[0] in lip_subsite.keys():
            update_cancer_episode(row, lip_subsite, 'lip_subsite')
        elif row2[0] in upper_and_lower_limbs_subsite.keys():
            update_cancer_episode(row, upper_and_lower_limbs_subsite, 'upper_and_lower_limbs_subsite')
        elif row2[0] in trunk_wall_subsite.keys():
            update_cancer_episode(row, trunk_wall_subsite, 'trunk_wall_subsite')
        elif row2[0] in intra_abdominal_subsite.keys():
            update_cancer_episode(row, intra_abdominal_subsite, 'intra_abdominal_subsite')
        elif row2[0] in intra_thoracic_subsite.keys():
            update_cancer_episode(row, intra_thoracic_subsite, 'intra_thoracic_subsite')
        elif row2[0] in genito_urinary_subsite.keys():
            update_cancer_episode(row, genito_urinary_subsite, 'genito_urinary_subsite')
        elif row2[0] in head_and_neck_subsite.keys():
            update_cancer_episode(row, head_and_neck_subsite, 'head_and_neck_subsite')
        elif row2[0] in breast_subsite.keys():
            update_cancer_episode(row, breast_subsite, 'breast_subsite')
        elif row2[0] in est_and_related_tumours.keys():
            update_cancer_episode(row, est_and_related_tumours, 'est_and_related_tumours')
        else:
            print('No histology found')
            continue

conn.commit()
conn2.commit()
curOMOP.close()
curIDEA.close()
        

### From Measurement to Cancer Episode

What should we do with depth? here it is ENUM but in the excel we have 2 types of depth, both floats.

In [ ]:
dict_concepts = {
    1634371: "GRADE_1",
    1634752: "GRADE_2",
    1633749: "GRADE_3",
    1635587: "GRADE_H",
    1634085: "GRADE_L",
    9189: "NEGATIVE",
    9191: "POSITIVE",
    45878602: "NOT_TESTED"
}


query= """
    SELECT id
    FROM patient;
"""

query2= """
    SELECT measurement_concept_id, value_as_number, measurement_event_id
    FROM omopcdm.measurement WHERE patient_id = %s AND measurement_concept_id IN (4159955, 36768664, 36768911, 36768749, 4227243, 3043849, 46236082, 3000965);
"""
curOMOP.execute(query)

results = curIDEA.fetchall()

for row in results:
    curIDEA.execute(query2, (row[0]))
    results2 = curIDEA.fetchall()
    for row2 in results2:
        idEpisode= dictKeys.get(row2[2])
        if row2[0] == 4159955:
            insert_query = """
                UPDATE cancer_episode
                SET grading = %s
                WHERE patient_id = %s
                AND id = %s;
            """
            value=dict_concepts.get(row2[1])
            curIDEA.execute(insert_query, (value, row[0], idEpisode))
        elif row2[0] == 36768664:
            insert_query = """
                UPDATE cancer_episode
                SET tumor_size = %s
                WHERE patient_id = %s
                AND id = %s;
            """
            curIDEA.execute(insert_query, (row2[1], row[0], idEpisode))
        elif row2[0] == 36768911:
            #Code for depth
        elif row2[0] == 36768749:
            #Code for depth
        elif row2[0] == 4227243:
            insert_query = """
                UPDATE cancer_episode
                SET biopsy_mitotic_count = %s
                WHERE patient_id = %s
                AND id = %s;
            """
            curIDEA.execute(insert_query, (row2[1], row[0]))
        elif row2[0] == 3043849:
            insert_query = """
                UPDATE cancer_episode
                SET plasmatic_ebv_dna_at_baseline = %s
                WHERE patient_id = %s
                AND id = %s;
            """
            curIDEA.execute(insert_query, (dict_concepts.get(row2[1]), row[0]))
        elif row2[0] == 46236082:
            insert_query = """
                UPDATE cancer_episode
                SET hpv_status = %s
                WHERE patient_id = %s
                AND id = %s;
            """
            curIDEA.execute(insert_query, (dict_concepts.get(row2[1]), row[0]))
        elif row2[0] == 3000965:
            insert_query = """
                UPDATE cancer_episode
                SET crpcreactive_protein_tested = %s
                WHERE patient_id = %s
                AND id = %s;
            """
            curIDEA.execute(insert_query, (dict_concepts.get(row2[1]), row[0]))
        else:
            print('No measurement found')
            continue
        
conn.commit()
conn2.commit()
curOMOP.close()
curIDEA.close()


            

### From Observation to Cancer Episode

I need to closely examine the cardinality and verify whether using the patient ID is appropriate.

In [ ]:
query= """
    SELECT id
    FROM patient;
"""

query2= """
    SELECT observation_concept_id, observation_event_id
    FROM omopcdm.observation WHERE patient_id = %s AND observation_concept_id = 37117814 AND obs_event_field_concept_id=1147082;
"""

curOMOP.execute(query)

results = curIDEA.fetchall()

for row in results:
    curIDEA.execute(query2, (row))
    results2 = curIDEA.fetchone()
    if results2:
        insert_query = """
            UPDATE cancer_episode
            SET radiotherapy_induced_sarcoma = %s
            WHERE patient_id = %s AND id= %s;
        """
        curIDEA.execute(insert_query, ('RADIATION_THERAPY_IND_CHANGE', row[0],dictKeys.get(results2[1])))
